In [1]:
from glob import glob
import pandas as pd
from pathlib import Path
import sys
from itertools import combinations, product
sys.path.append('../')

import matplotlib.pyplot as plt
from tqdm import tqdm
from astropy.table import unique, join
import papermill as pm


In [2]:
from uvot import filters
from xrt import curves
from source_names_dict import source_names_dict, source_names_w_counterparts, source_names_readable
from get_closest_srcreg import get_src_region_dict
from uvot import read_uvotsource, calc_uvot_flags, filters, filter_colors, filter_markers
from xrt import load_xrt, rename_xrt_cols, curves
from lightcurve import LightCurve

In [3]:
all_df = []
for f in tqdm(glob('../lightcurves/correlation_output/*/tables/*table_prop*')):
    sp = f.split('/')[-1].split(',')
    simbad_name = sp[0]
    xrt_curve   = sp[1]
    uvot_filter = sp[2]
    include_bad = sp[3]
    include_UL  = sp[4]
    
    df = pd.read_csv(f)
    df['simbad_name'] = simbad_name
    df['xrt_curve'] = xrt_curve
    df['uvot_filter'] = uvot_filter
    df['include_bad'] = include_bad
    df['include_UL']  = include_UL
    
    all_df.append(df)
df_prop = pd.concat(all_df)
df_prop


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                    | 913/1019 [00:04<00:00, 191.63it/s]

KeyboardInterrupt



In [ ]:
all_res = []
for f in tqdm(glob('../lightcurves/correlation_output/*/tables/*table_corr*')):
    sp = f.split('/')[-1].split(',')
    simbad_name = sp[0]
    xrt_curve = sp[1]
    uvot_filter = sp[2]
    include_bad = sp[3]
    include_UL  = sp[4]
    
    # print(simbad_name, xrt_curve, uvot_filter)
    
    df = pd.read_csv(f)
    r_mean = df['r'].mean()
    r_std  = df['r'].std()
    m_mean = df['m'].mean()
    m_std  = df['m'].std()
    c_mean = df['c'].mean()
    c_std  = df['c'].std()
    
    res = {}
    res['simbad_name'] = simbad_name
    res['xrt_curve'] = xrt_curve
    res['uvot_filter'] = uvot_filter
    res['include_bad'] = include_bad
    res['include_UL'] = include_UL
    
    res['r_mean'] = r_mean
    res['r_std'] = r_std
    res['m_mean'] = m_mean
    res['m_std'] = m_std
    res['c_mean'] = c_mean
    res['c_std'] = c_std
    
    all_res.append(res)
df = pd.DataFrame(all_res)
df

In [ ]:
exclude = ['SWIFT_J0243.6+6124', 'NOVA_Cyg_1989']

nsig     = 2
r_mu     = df['r_mean'].mean()
r_std    = df['r_mean'].std()
r_sig_up = r_mu + nsig*r_std
r_sig_lo = r_mu - nsig*r_std


df['nsig'] = abs((df['r_mean'] - r_mu) / r_std)

plt.figure(figsize=(30,30), facecolor='white')
for i, s in enumerate(df['simbad_name'].unique()):
    if s in exclude:
        continue
    sub = df[df['simbad_name'] == s]
    plt.scatter(sub['m_mean'], sub['r_mean'], marker=f'${i}$', label=s, s=100)
    #plt.errorbar(sub['m_mean'], sub['r_mean'], xerr=sub['m_std'], yerr=sub['r_std'], marker=f'${i}$', label=s, markersize=10)
plt.xlabel('m_mean')
plt.ylabel('r_mean / r_std')

plt.xscale('symlog')
#plt.ylim(-5,5)
plt.xlim(-10,10)
plt.legend()
#plt.ylim(-10,10)


In [ ]:
sub_nsig = df[(df['r_mean'] > r_sig_up) | (df['r_mean'] < r_sig_lo)]

In [ ]:
df_prop_tab_5_sig = df_prop[df_prop['name'] == 'tab_5_sig']

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
pd.merge(sub_nsig, df_prop_tab_5_sig, on=['simbad_name','xrt_curve','uvot_filter','include_bad','include_UL'])